In [27]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.5 MB/s eta 0:00:00


In [15]:
# Colab 환경에서 진행하기 때문에 google drive 연결

from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads #Multi-head Attention에서는 query, key, value를 바로 사용하는 것이 아닌 h번의 Linear projection을 따라 서로 다른 representation의 조합으로부터 Attention을 계산하는 방법이다. 
        self.query_dense = layers.Dense(embed_dim) #쿼리
        self.key_dense = layers.Dense(embed_dim) #키
        self.value_dense = layers.Dense(embed_dim) #밸류
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True) #Q와 V를 곱한다.
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32) #텐서를 새로운 자료형으로 변환합니다.(tf.shape(key)[-1] = 차원)
        scaled_score = score / tf.math.sqrt(dim_key) #Sclae 작업, K차원의 루트값으로
        weights = tf.nn.softmax(scaled_score, axis=-1) #Softmax
        output = tf.matmul(weights, value) #V 곱하기
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim)) #Multihead Attention
        return tf.transpose(x, perm=[0, 2, 1, 3]) #x를 전치합니다. perm에 따라 차원의 순서를 구성합니다.

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0] #배치크기 
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim) => tf.transpose(x, perm=[0, 2, 1, 3])의 결과
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value) #Self Attention
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output

## Transformer Layer

In [2]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs) #Multihead Attn 블록
        attn_output = self.dropout1(attn_output, training=training) #드롭아웃
        out1 = self.layernorm1(inputs + attn_output) #LM + Residual
        ffn_output = self.ffn(out1) #FF 블록
        ffn_output = self.dropout2(ffn_output, training=training) #드롭아웃
        return self.layernorm2(out1 + ffn_output) #LM + Residual

## Embedding Layer

In [3]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, emded_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=emded_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=emded_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1) #포지션 정보
        positions = self.pos_emb(positions) #포지션 임베딩
        x = self.token_emb(x) #토큰임베딩
        return x + positions #합치기

## 데이터셋 준비 및 전처리

In [108]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/Deep_Learning/NLP/finance_sentiment_corpus-main/finance_data.csv',encoding='utf-8-sig')

In [109]:
data.head()

,labels,sentence,kor_sentence
0,neutral,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,neutral,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,negative,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,positive,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,positive,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [110]:
def sentiment_analysis(data):
  if data == 'negative':
    return 0
  elif data == 'neutral':
    return 1
  else:
    return 2

In [111]:
data['labels_tr'] = data['labels'].apply(lambda x: sentiment_analysis(x))

In [112]:
data = data[['kor_sentence','labels_tr']]
data = [[i, j] for i, j in zip(data['kor_sentence'], data['labels_tr'])]

In [114]:
data_dir = '/content/drive/MyDrive/NLP_Practice/'

In [115]:
import sentencepiece as spm

# vocab loading
vocab_file = f"{data_dir}kowiki.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

True

In [116]:
import numpy as np

arr_data = np.array(data)

In [117]:
lst = []
for idx in range(len(arr_data)):
  vec = vocab.encode(arr_data[idx,0])
  lst.append(vec)

In [156]:
arr_lst = np.array(lst).reshape(-1,).tolist()
arr_label = np.array(data)[:,1].tolist()

<ipython-input-156-5cadd6c037c5>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_lst = np.array(lst).reshape(-1,).tolist()


In [157]:
arr_label = list( map(lambda x: int(x) , arr_label ) )

In [158]:
from sklearn.model_selection import train_test_split


x_train, x_val, y_train, y_val = train_test_split(arr_lst,arr_label,test_size = 0.2, random_state=0) #데이터셋 로딩
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")

3876 Training sequences
970 Validation sequences


In [167]:
x_train = keras.preprocessing.sequence.pad_sequences(x_train,maxlen=maxlen)#패딩
x_val = keras.preprocessing.sequence.pad_sequences(x_val,maxlen=maxlen)#패딩

In [168]:
y_train = np.array(y_train)
y_val = np.array(y_val)

## 모델 구축

In [169]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,)) #처음 입력
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim) #객체 생성
x = embedding_layer(inputs)  #포지셔널 임베딩
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim) #객체 생성
x = transformer_block(x) #트랜스포머 
x = layers.GlobalAveragePooling1D()(x) #Average Pooling
x = layers.Dropout(0.1)(x) #드롯아웃
x = layers.Dense(20, activation="relu")(x) #FFNN
x = layers.Dropout(0.1)(x) #드롭아웃
outputs = layers.Dense(3, activation="softmax")(x) #Softmax

model = keras.Model(inputs=inputs, outputs=outputs) #모델 생성

## 학습

In [170]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

history = model.fit(
    x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val)
)

Epoch 1/5
122/122 [==============================] - 11s 66ms/step - loss: 0.9359 - accuracy: 0.5800 - val_loss: 0.9267 - val_accuracy: 0.6021
Epoch 2/5
122/122 [==============================] - 7s 60ms/step - loss: 0.8450 - accuracy: 0.6373 - val_loss: 0.7569 - val_accuracy: 0.6711
Epoch 3/5
122/122 [==============================] - 7s 54ms/step - loss: 0.6379 - accuracy: 0.7371 - val_loss: 0.6509 - val_accuracy: 0.7278
Epoch 4/5
122/122 [==============================] - 7s 60ms/step - loss: 0.4657 - accuracy: 0.8313 - val_loss: 0.6602 - val_accuracy: 0.7258
Epoch 5/5
122/122 [==============================] - 7s 54ms/step - loss: 0.3243 - accuracy: 0.8880 - val_loss: 0.7552 - val_accuracy: 0.7629


## 성능측정

In [171]:
#모델 정보 출력
model.summary() 

#성능 측정
test_loss,test_acc=model.evaluate(x_val,y_val)
print("Test_acc: ",test_acc)

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 200)]             0         
                                                                 
 token_and_position_embeddin  (None, 200, 32)          646400    
 g_10 (TokenAndPositionEmbed                                     
 ding)                                                           
                                                                 
 transformer_block_10 (Trans  (None, 200, 32)          6464      
 formerBlock)                                                    
                                                                 
 global_average_pooling1d_10  (None, 32)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dropout_42 (Dropout)        (None, 32)                0  